<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/3_version_coklu_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# DOSYA ADI: egitim_stacking.py
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import joblib # Sklearn/XGBoost modellerini kaydetmek için
import xgboost as xgb
import lightgbm as lgb

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

# --- 1. ADIM: VERİ ÜRETİMİ (Aynı kalıyor) ---
print(">> Veri üretiliyor (50.000 Adet)...") # Hız için sayıyı 50k yaptım, artırabilirsin.

def veri_uret(n):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Sayısal özellikler
        risk = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        cons = np.random.uniform(0, 10)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        # Etiketleme (Biraz gürültü eklenmiş)
        score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
        score += np.random.normal(0, 0.05)
        label = 1 if score > 0.65 else 0

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    # DataFrame'e çevirelim (Tabular modeller için kolaylık olsun)
    df_feat = pd.DataFrame(bio_feats, columns=['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight'])
    return dna_seqs, prot_seqs, df_feat, np.array(labels)

dna_data, prot_data, X_num, y = veri_uret(50000)

# --- 2. ADIM: ÖN İŞLEME ---
# Tokenizer
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_data)
X_dna = pad_sequences(dna_tok.texts_to_sequences(dna_data), maxlen=11, padding='post')

prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_data)
X_prot = pad_sequences(prot_tok.texts_to_sequences(prot_data), maxlen=11, padding='post')

# --- 3. ADIM: STACKING İÇİN VERİ BÖLME ---
# Stacking'de en önemli kural: Meta-Model, Base modellerin eğitimde görmediği veriyle eğitilmelidir!
# Bu yüzden veriyi 3 parçaya bölüyoruz:
# 1. Base_Train (%60): Temel modelleri eğitmek için.
# 2. Meta_Train (%20): Temel modellerin tahmin üretip Meta modelin eğitilmesi için.
# 3. Test (%20): Nihai başarıyı ölçmek için.

# Önce Test'i ayıralım
X_dna_temp, X_dna_test, X_prot_temp, X_prot_test, X_num_temp, X_num_test, y_temp, y_test = train_test_split(
    X_dna, X_prot, X_num, y, test_size=0.2, random_state=42
)

# Kalanı Base ve Meta olarak ayıralım
X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna_temp, X_prot_temp, X_num_temp, y_temp, test_size=0.25, random_state=42 # 0.25 * 0.8 = 0.20 Toplamın %20'si
)

print(f">> Veri Bölündü: Base Eğitim: {len(y_base)}, Meta Eğitim: {len(y_meta)}, Test: {len(y_test)}")

# --- 4. ADIM: BASE MODELLERİN EĞİTİMİ ---

# === MODEL 1: DERİN ÖĞRENME (CNN) ===
# Hem dizileri hem sayısal veriyi kullanır
print("\n>> 1/3 Derin Öğrenme Modeli (CNN) Eğitiliyor...")
in_dna = Input(shape=(11,))
emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_dna))

in_prot = Input(shape=(11,))
emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_prot))

in_num_layer = Input(shape=(6,))
x3 = BatchNormalization()(Dense(16, activation='relu')(in_num_layer))

merged = Concatenate()([x1, x2, x3])
z = Dropout(0.3)(Dense(32, activation='relu')(merged))
out = Dense(1, activation='sigmoid')(z)

model_cnn = Model(inputs=[in_dna, in_prot, in_num_layer], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cnn.fit([X_dna_base, X_prot_base, X_num_base], y_base, epochs=5, batch_size=32, verbose=0)
print("   ✅ CNN Tamamlandı.")

# === MODEL 2: XGBOOST ===
# Sadece Sayısal veriyi sever (Dizi verisi vermiyoruz, sadece X_num)
print(">> 2/3 XGBoost Modeli Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X_num_base, y_base)
print("   ✅ XGBoost Tamamlandı.")

# === MODEL 3: LIGHTGBM ===
# Sadece Sayısal veriyi sever
print(">> 3/3 LightGBM Modeli Eğitiliyor...")
model_lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, verbose=-1)
model_lgbm.fit(X_num_base, y_base)
print("   ✅ LightGBM Tamamlandı.")


# --- 5. ADIM: META-LEARNER (STACKING) EĞİTİMİ ---
print("\n>> Meta-Learner için Tahminler Toplanıyor...")

# Meta seti üzerinde modellerin "fikirlerini" (olasılıklarını) alıyoruz
pred_cnn_meta = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
pred_xgb_meta = model_xgb.predict_proba(X_num_meta)[:, 1] # 1 olma olasılığı
pred_lgbm_meta = model_lgbm.predict_proba(X_num_meta)[:, 1]

# Bu tahminleri yeni bir DataFrame'de birleştiriyoruz
# Bu tablo: [CNN_Fikri, XGB_Fikri, LGBM_Fikri] -> Gerçek_Label
X_stack_meta = np.column_stack((pred_cnn_meta, pred_xgb_meta, pred_lgbm_meta))

print(">> Meta-Learner (Logistic Regression) Eğitiliyor...")
meta_model = LogisticRegression()
meta_model.fit(X_stack_meta, y_meta)

# --- 6. ADIM: TEST PERFORMANSI ---
print("\n>> FİNAL TESTİ (Hiç Görülmemiş Veriyle)...")

# Test seti için tahminleri al
p1 = model_cnn.predict([X_dna_test, X_prot_test, X_num_test], verbose=0).flatten()
p2 = model_xgb.predict_proba(X_num_test)[:, 1]
p3 = model_lgbm.predict_proba(X_num_test)[:, 1]

# Birleştir
X_stack_test = np.column_stack((p1, p2, p3))

# Meta model karar versin
final_preds = meta_model.predict(X_stack_test)

print(f"STACKING F1 SKORU: {f1_score(y_test, final_preds):.4f}")
print(f"STACKING ACCURACY: {accuracy_score(y_test, final_preds):.4f}")
print("Meta Model Ağırlıkları (Hangi modele ne kadar güvendi?):")
print(f"CNN: {meta_model.coef_[0][0]:.2f}, XGB: {meta_model.coef_[0][1]:.2f}, LGBM: {meta_model.coef_[0][2]:.2f}")

# --- 7. ADIM: KAYIT ---
print("\n>> Tüm modeller kaydediliyor...")
import os
if not os.path.exists('model_stack'): os.makedirs('model_stack')

# 1. Base Modeller
model_cnn.save('model_stack/base_cnn.h5')
joblib.dump(model_xgb, 'model_stack/base_xgb.pkl')
joblib.dump(model_lgbm, 'model_stack/base_lgbm.pkl')

# 2. Meta Model
joblib.dump(meta_model, 'model_stack/meta_model.pkl')

# 3. Tokenizerlar
with open('model_stack/dna_tok.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok.pickle', 'wb') as f: pickle.dump(prot_tok, f)

# 4. Test için örnek bir veri dosyası kaydedelim (doğrulama kodunda kullanmak için)
# Gerçek veriler ve sonuçları birleştirip kaydediyoruz
test_df = X_num_test.copy()
test_df['DNA_Dizisi'] = dna_tok.sequences_to_texts(X_dna_test)
test_df['DNA_Dizisi'] = test_df['DNA_Dizisi'].apply(lambda x: x.replace(' ', '').upper())
test_df['Protein_Dizisi'] = prot_tok.sequences_to_texts(X_prot_test)
test_df['Protein_Dizisi'] = test_df['Protein_Dizisi'].apply(lambda x: x.replace(' ', '').upper())
test_df['Gercek_Deger'] = y_test

test_df.to_csv('test_verileri_stacking.csv', index=False)
print("✅ Her şey kaydedildi! 'model_stack' klasörünü kontrol edin.")

>> Veri üretiliyor (50.000 Adet)...
>> Veri Bölündü: Base Eğitim: 30000, Meta Eğitim: 10000, Test: 10000

>> 1/3 Derin Öğrenme Modeli (CNN) Eğitiliyor...
   ✅ CNN Tamamlandı.
>> 2/3 XGBoost Modeli Eğitiliyor...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [00:51:20] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


   ✅ XGBoost Tamamlandı.
>> 3/3 LightGBM Modeli Eğitiliyor...
   ✅ LightGBM Tamamlandı.

>> Meta-Learner için Tahminler Toplanıyor...
>> Meta-Learner (Logistic Regression) Eğitiliyor...

>> FİNAL TESTİ (Hiç Görülmemiş Veriyle)...


STACKING F1 SKORU: 0.8273
STACKING ACCURACY: 0.8954
Meta Model Ağırlıkları (Hangi modele ne kadar güvendi?):
CNN: 5.39, XGB: 2.23, LGBM: -0.54

>> Tüm modeller kaydediliyor...
✅ Her şey kaydedildi! 'model_stack' klasörünü kontrol edin.


In [37]:
# DOSYA ADI: test_stacking.py
import pandas as pd
import numpy as np
import pickle
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(">> Modeller ve Sözlükler yükleniyor...")

# 1. YÜKLEME İŞLEMLERİ
# Sözlükler
with open('model_stack/dna_tok.pickle', 'rb') as f: dna_tok = pickle.load(f)
with open('model_stack/prot_tok.pickle', 'rb') as f: prot_tok = pickle.load(f)

# Base Modeller
model_cnn = load_model('model_stack/base_cnn.h5')
model_xgb = joblib.load('model_stack/base_xgb.pkl')
model_lgbm = joblib.load('model_stack/base_lgbm.pkl')

# Meta Model
meta_model = joblib.load('model_stack/meta_model.pkl')

print("✅ Sistem Hazır.")

# 2. TEST VERİSİNDEN RASTGELE HASTA ÇEK
try:
    df = pd.read_csv('test_verileri_stacking.csv')
    hasta = df.sample(1).iloc[0]
except:
    print("Test verisi bulunamadı, lütfen önce eğitimi çalıştırın.")
    exit()

# Gelen Ham Veriler
raw_dna = hasta['DNA_Dizisi']
raw_prot = hasta['Protein_Dizisi']
raw_feats = hasta[['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight']].values.reshape(1, -1) # (1, 6) formatında
gercek_durum = int(hasta['Gercek_Deger'])


# 3. VERİYİ HAZIRLA
# CNN için sequence hazırlığı (Bunlar zaten int gelir, sorun çıkarmaz)
seq_dna = pad_sequences(dna_tok.texts_to_sequences([raw_dna]), maxlen=11, padding='post')
seq_prot = pad_sequences(prot_tok.texts_to_sequences([raw_prot]), maxlen=11, padding='post')

# *** KRİTİK DÜZELTME BURADA ***
# raw_feats nesne (object) tipinde kalmış olabilir, onu zorla sayıya (float32) çeviriyoruz.
raw_feats = np.asarray(raw_feats).astype('float32')

# XGB/LGBM için features (Pandas DataFrame olarak vermek uyarıları engeller)
feat_df = pd.DataFrame(raw_feats, columns=['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight'])

# 4. UZMANLARA DANIŞ (BASE MODELS PREDICTION)
print("\n>> 🧠 Uzman Modeller Düşünüyor...")

# Uzman 1: CNN (Olasılık veriyor)
# Artık raw_feats float32 olduğu için hata vermeyecek
prob_cnn = model_cnn.predict([seq_dna, seq_prot, raw_feats], verbose=0)[0][0]

# Uzman 2: XGBoost (Olasılık veriyor - sınıf 1 için)
prob_xgb = model_xgb.predict_proba(feat_df)[0][1]

# Uzman 3: LightGBM (Olasılık veriyor - sınıf 1 için)
prob_lgbm = model_lgbm.predict_proba(feat_df)[0][1]

print(f"   - CNN Görüşü     : %{prob_cnn*100:.2f} Hasta Olabilir")
print(f"   - XGBoost Görüşü : %{prob_xgb*100:.2f} Hasta Olabilir")
print(f"   - LightGBM Görüşü: %{prob_lgbm*100:.2f} Hasta Olabilir")

#
# 4. UZMANLARA DANIŞ (BASE MODELS PREDICTION)
print("\n>> 🧠 Uzman Modeller Düşünüyor...")



# 5. PATRONA DANIŞ (META LEARNER PREDICTION)
# Uzman görüşlerini bir vektörde topla: [CNN, XGB, LGBM]
stack_vector = np.array([[prob_cnn, prob_xgb, prob_lgbm]])

# Patron karar versin
final_prob = meta_model.predict_proba(stack_vector)[0][1]
final_karar = 1 if final_prob > 0.5 else 0

# 6. RAPOR
print("\n" + "="*40)
print("🏥 STACKING TANI SİSTEMİ SONUCU")
print("="*40)
print(f"DNA Dizisi       : {raw_dna}")
print(f"Biyo-İşaretler   : Risk={raw_feats[0][0]:.2f}, Ağırlık={raw_feats[0][5]:.1f}")
print("-" * 40)
print(f"GERÇEK DURUM     : {'HASTA ⚠️' if gercek_durum==1 else 'SAĞLIKLI ✅'}")
print(f"SİSTEM TAHMİNİ   : {'HASTA ⚠️' if final_karar==1 else 'SAĞLIKLI ✅'}")
print(f"Güven Skoru      : %{final_prob*100:.2f}")
print("-" * 40)

if final_karar == gercek_durum:
    print("SONUÇ: 🏆 TEBRİKLER! Sistem doğru bildi.")
else:
    print("SONUÇ: ❌ HATA. Sistem yanıldı.")

>> Modeller ve Sözlükler yükleniyor...
✅ Sistem Hazır.

>> 🧠 Uzman Modeller Düşünüyor...
   - CNN Görüşü     : %0.03 Hasta Olabilir
   - XGBoost Görüşü : %0.19 Hasta Olabilir
   - LightGBM Görüşü: %0.30 Hasta Olabilir

>> 🧠 Uzman Modeller Düşünüyor...

🏥 STACKING TANI SİSTEMİ SONUCU
DNA Dizisi       : TAGCACGGTAG
Biyo-İşaretler   : Risk=0.12, Ağırlık=17.4
----------------------------------------
GERÇEK DURUM     : SAĞLIKLI ✅
SİSTEM TAHMİNİ   : SAĞLIKLI ✅
Güven Skoru      : %1.94
----------------------------------------
SONUÇ: 🏆 TEBRİKLER! Sistem doğru bildi.
